<a href="https://colab.research.google.com/github/aicrashcoursewinter24/ailab1BrianMann/blob/lab3/Copy_of_1_pretrained_vectors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part I: Pre-trained embeddings

We are going to build some PyTorch models that are commonly used for text classification.  We also need to build out some infrastructure to run these models.

Once we have the models and the boilerplate stuff out of the way, we can see the impact of pre-trained embeddings for classification tasks. Pre-training methods like word2vec are context-limited language models whose goal is to predict a word given a fixed context, or a fixed context given a word. Pre-trained embeddings are particularly useful for smaller datasets.

Most of this code is inspired or derived from [Baseline](https://github.com/dpressel/baseline/) (Pressel et al, 2018), an open source project for building and evaluating NLP models across a variety of NLP tasks.  For this tutorial, we will only concern ourselves with Text Classification using a few useful models.

## Word Embeddings in NLP

We start our models with what are called "one-hot" vectors.  This is notionally a sparse vector with length `|V|` where V is our vocabulary, and where only the word representated at this temporal location is a 1.  The rest are zeros.

![onehot](https://www.tensorflow.org/images/feature_columns/categorical_column_with_vocabulary.jpg)

These vectors are not truly represented as a vector, but as an array of indices (in PyTorch, they are `torch.LongTensor`s), one for each word's index in the vocab.  This representation is not particularly helpful in DNNs since we want continuous representations for each word.

![indices](https://www.tensorflow.org/images/feature_columns/categorical_column_with_identity.jpg)

The general idea of an embedding is that we want to project from a large one-hot vector to a compact, distributed representation with smaller dimensionality.  We can look at this as a matrix multiply between a one-hot vector `|V|` and a weight matrix to a lower dimension of size `|D|`.  Since only a single vector value in the one-hot vector is on at a time, this matrix multiply is simplified to an address lookup in that matrix.

![LUT](https://cdn-images-1.medium.com/max/800/1*fZj1Hk1mhS5pIMv3ZrpLYw.png)


In PyTorch, this is called an `nn.Embedding`.  In fact, in Torch7, this was called a `nn.LookupTable` which may have actually been a better name, but which seems to have fallen out of favor in DNN toolkits.  In this tutorial we are going to refer to multiple types of embeddings, and in this case, we are referring to word vectors, which are typically lookup table embeddings.

Embeddings make up lowest layer of a typical DNN for text and we will feed their output to some pooling mechanism yielding a fixed length representation, followed by some number of fully connected layers.

### Pre-training with Word2Vec

There has been a large amount of research that has gone into building distributed representations for words through pre-training. Some widely used algorithms in NLP include Word2Vec, GloVe and fastText.  For instance, word2vec is actually 2 different algorithms with 2 different objectives.  They can be thought of a fixed context window non-causal LMs, but they are shallow models and extremely fast to train

![word2vec](https://deeplearning4j.org/img/word2vec_diagrams.png)


* **CBOW objective** given all words in a fixed context window except the middle word, predict the middle word
* **SkipGram objective**: given a word in a fixed context window, predict all other words in that window

Once we have trained these models, the learned distributed representation matrix can be plugged right in as our embedding weights and this often improves the model significantly.

Before we begin, we will download some data that can be used for our experiments

In [1]:
!wget https://www.dropbox.com/s/7jyi4pi894bh2qh/sst2.tar.gz?dl=1
!tar -xzf 'sst2.tar.gz?dl=1'

!wget https://www.dropbox.com/s/08km2ean8bkt7p3/trec.tar.gz?dl=1
!tar -xzf 'trec.tar.gz?dl=1'

!wget https://www.dropbox.com/s/699kgut7hdb5tg9/GoogleNews-vectors-negative300.bin.gz?dl=1
!mv 'GoogleNews-vectors-negative300.bin.gz?dl=1' GoogleNews-vectors-negative300.bin.gz
!gunzip GoogleNews-vectors-negative300.bin.gz

--2024-01-10 21:11:07--  https://www.dropbox.com/s/7jyi4pi894bh2qh/sst2.tar.gz?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/7jyi4pi894bh2qh/sst2.tar.gz [following]
--2024-01-10 21:11:07--  https://www.dropbox.com/s/dl/7jyi4pi894bh2qh/sst2.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uceb7edfbbd1bd8902282ffa5c4a.dl.dropboxusercontent.com/cd/0/get/CLE2WRgOJpVvV_RnGCsmKtCWU4PRwB284XAnBoDU9dGlQENDBHD7Iay7OdHQUPr30L9MywYowqtD4NQ-kaROzzvQHaT9SOkkfJVvXHnG9vnvbIi8jqVrRDEqarqj_bPDLBawkYqMwELe-4tQRrXplJli/file?dl=1# [following]
--2024-01-10 21:11:07--  https://uceb7edfbbd1bd8902282ffa5c4a.dl.dropboxusercontent.com/cd/0/get/CLE2WRgOJpVvV_RnGCsmKtCWU4PRwB284XAnBoDU9dGlQENDBHD7Iay7OdHQUPr30L9MywYowqtD4NQ-kaROzzvQHaT9SOkkfJVvXHn

## First, lets do some fun stuff

We will start by building out some models that we will reuse later in the tutorial.  First, we will build a convolutional neural network (CNN) that can classify text. Basically CNNs learn a kernel that can be used to filter images or text.  An example of 2D filtering*:

![2D filtering](https://cdn-images-1.medium.com/max/1600/0*9J3MK1gd2zrFDzDN.gif)

In the case of text filtering, we have a one-dimensional filter operation like this*:

![1D filtering](http://cs231n.github.io/assets/cnn/stride.jpeg)

This type of model has been used often in text, including by [Collobert et al 2011](https://ronan.collobert.com/pub/matos/2011_nlp_jmlr.pdf), but we will implement a multiple parallel filter variation of this introduced by [Kim 2014](https://www.aclweb.org/anthology/D14-1181).

### Convolutional Neural Network for Text Classification

We are using PyTorch, so every layer we have is going to inherit `nn.Module`.

#### Convolutions (actually cross correlations)

The first characteristic of this model is that we will have multiple convolutional filter lengths, and some number of filters associated with each length.  For each filter of length `K` convolved with a signal of length `T`, the output signal will be `T - K + 1`.  To handle the ends of the signal where the filter is hanging off (e.g. centered at 0), we will add some zero-padding.  So if we have a filter of length `K=3`, we want to zero-pad the temporal signal by a single pad value on both ends of the signal.

We are going to support multiple parallel filters, so we can add a `torch.nn.Conv1d` for each filter length, followed by a `torch.nn.ReLU` activation layer.  Since we have more than one of these, we will create a `nn.ModuleList` to track them.  When we call `forward()`, the data will be oriented as $$B \times C \times T$$ where `B` is the batch size,  `C` is the number of hidden units and `T` is the temporal length of the vector.

#### Pooling

Both of the papers mentioned above do max-over-time pooling over the features.  For each feature map in the vector, we simply select the maximum value for that feature map and concatenate all of these together.  Our $$B \times C \times T$$ vector is then going to be reduced along the time dimension to $$B \times C$$

*Images courtesy of http://cs231n.github.io/convolutional-networks/

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import List, Tuple
import os
import io
import re
import codecs
from collections import Counter
from torch.utils.data import DataLoader, TensorDataset

In [3]:
class ParallelConv(nn.Module):

    def __init__(self, input_dims, filters, dropout=0.5):
        super().__init__()
        convs = []
        self.output_dims = sum([t[1] for t in filters])
        for (filter_length, output_dims) in filters:
            pad = filter_length//2
            conv = nn.Sequential(
                nn.Conv1d(input_dims, output_dims, filter_length, padding=pad),
                nn.ReLU()
            )
            convs.append(conv)
        # Add the module so its managed correctly
        self.convs = nn.ModuleList(convs)
        self.conv_drop = nn.Dropout(dropout)


    def forward(self, input_bct):
        mots = []
        for conv in self.convs:
            # In Conv1d, data BxCxT, max over time
            conv_out = conv(input_bct)
            mot, _ = conv_out.max(2)
            mots.append(mot)
        mots = torch.cat(mots, 1)
        return self.conv_drop(mots)

class ConvClassifier(nn.Module):

    def __init__(self, embeddings, num_classes, embed_dims,
                 filters=[(2, 100), (3, 100), (4, 100)],
                 dropout=0.5, hidden_units=[]):
        super().__init__()
        self.embeddings = embeddings
        self.dropout = nn.Dropout(dropout)
        self.convs = ParallelConv(embed_dims, filters, dropout)

        input_units = self.convs.output_dims
        output_units = self.convs.output_dims
        sequence = []
        for h in hidden_units:
            sequence.append(self.dropout(nn.Linear(input_units, h)))
            input_units = h
            output_units = h

        sequence.append(nn.Linear(output_units, num_classes))
        self.outputs = nn.Sequential(*sequence)

    def forward(self, inputs):
        one_hots, lengths = inputs
        embed = self.dropout(self.embeddings(one_hots))
        embed = embed.transpose(1, 2).contiguous()
        hidden = self.convs(embed)
        linear = self.outputs(hidden)
        return F.log_softmax(linear, dim=-1)


### LSTM Model

Our second model that we will explore uses Long Short-Term Memory (LSTM) units, which are a form of Recurrent Neural Networks.  These models tend to perform extremely well on NLP tasks.  Text classification is a simple case, where we give our inputs and take the final LSTM output as a form of pooling.  That looks like the **Many-to-One** image in this taxonomy from [Andrej Karpathy's 2015 blog post on using RNNs for character-level language modeling](https://karpathy.github.io/2015/05/21/rnn-effectiveness/)

![Many-to-one LSTM](https://karpathy.github.io/assets/rnn/diags.jpeg)

In [4]:
class LSTMClassifier(nn.Module):

    def __init__(self, embeddings, num_classes, embed_dims, rnn_units, rnn_layers=1, dropout=0.5, hidden_units=[]):
        super().__init__()
        self.embeddings = embeddings
        self.dropout = nn.Dropout(dropout)
        self.rnn = torch.nn.LSTM(embed_dims,
                                 rnn_units,
                                 rnn_layers,
                                 dropout=dropout,
                                 bidirectional=False,
                                 batch_first=False)
        nn.init.orthogonal_(self.rnn.weight_hh_l0)
        nn.init.orthogonal_(self.rnn.weight_ih_l0)
        sequence = []
        input_units = rnn_units
        output_units = rnn_units
        for h in hidden_units:
            sequence.append(nn.Linear(input_units, h))
            input_units = h
            output_units = h

        sequence.append(nn.Linear(output_units, num_classes))
        self.outputs = nn.Sequential(*sequence)


    def forward(self, inputs):
        one_hots, lengths = inputs
        embed = self.dropout(self.embeddings(one_hots))
        embed = embed.transpose(0, 1)
        packed = torch.nn.utils.rnn.pack_padded_sequence(embed, lengths.tolist())
        _, hidden = self.rnn(packed)
        hidden = hidden[0].view(hidden[0].shape[1:])
        linear = self.outputs(hidden)
        return F.log_softmax(linear, dim=-1)


## Training our model

To set our model up for training (and evaluation), we need a loss function, some metrics, and an optimizer, along with some training data.

### Defining Metrics

For classification problems, most things we would like to know can be defined in terms of a confusion matrix.

![Confusion Matrix](https://scikit-learn.org/stable/_images/sphx_glr_plot_confusion_matrix_001.png)

The class below implements a confusion matrix and provides metrics associated using it.  This implementation is taken from verbatim from Baseline (https://github.com/dpressel/baseline/blob/master/python/baseline/confusion.py)

In [5]:

class ConfusionMatrix:
    """Confusion matrix with metrics

    This class accumulates classification output, and tracks it in a confusion matrix.
    Metrics are available that use the confusion matrix
    """
    def __init__(self, labels):
        """Constructor with input labels

        :param labels: Either a dictionary (`k=int,v=str`) or an array of labels
        """
        if type(labels) is dict:
            self.labels = []
            for i in range(len(labels)):
                self.labels.append(labels[i])
        else:
            self.labels = labels
        nc = len(self.labels)
        self._cm = np.zeros((nc, nc), dtype=np.int)

    def add(self, truth, guess):
        """Add a single value to the confusion matrix based off `truth` and `guess`

        :param truth: The real `y` value (or ground truth label)
        :param guess: The guess for `y` value (or assertion)
        """

        self._cm[truth, guess] += 1

    def __str__(self):
        values = []
        width = max(8, max(len(x) for x in self.labels) + 1)
        for i, label in enumerate([''] + self.labels):
            values += ["{:>{width}}".format(label, width=width+1)]
        values += ['\n']
        for i, label in enumerate(self.labels):
            values += ["{:>{width}}".format(label, width=width+1)]
            for j in range(len(self.labels)):
                values += ["{:{width}d}".format(self._cm[i, j], width=width + 1)]
            values += ['\n']
        values += ['\n']
        return ''.join(values)

    def save(self, outfile):
        ordered_fieldnames = OrderedDict([("labels", None)] + [(l, None) for l in self.labels])
        with open(outfile, 'w') as f:
            dw = csv.DictWriter(f, delimiter=',', fieldnames=ordered_fieldnames)
            dw.writeheader()
            for index, row in enumerate(self._cm):
                row_dict = {l: row[i] for i, l in enumerate(self.labels)}
                row_dict.update({"labels": self.labels[index]})
                dw.writerow(row_dict)

    def reset(self):
        """Reset the matrix
        """
        self._cm *= 0

    def get_correct(self):
        """Get the diagonals of the confusion matrix

        :return: (``int``) Number of correct classifications
        """
        return self._cm.diagonal().sum()

    def get_total(self):
        """Get total classifications

        :return: (``int``) total classifications
        """
        return self._cm.sum()

    def get_acc(self):
        """Get the accuracy

        :return: (``float``) accuracy
        """
        return float(self.get_correct())/self.get_total()

    def get_recall(self):
        """Get the recall

        :return: (``float``) recall
        """
        total = np.sum(self._cm, axis=1)
        total = (total == 0) + total
        return np.diag(self._cm) / total.astype(float)

    def get_support(self):
        return np.sum(self._cm, axis=1)

    def get_precision(self):
        """Get the precision
        :return: (``float``) precision
        """

        total = np.sum(self._cm, axis=0)
        total = (total == 0) + total
        return np.diag(self._cm) / total.astype(float)

    def get_mean_precision(self):
        """Get the mean precision across labels

        :return: (``float``) mean precision
        """
        return np.mean(self.get_precision())

    def get_weighted_precision(self):
        return np.sum(self.get_precision() * self.get_support())/float(self.get_total())

    def get_mean_recall(self):
        """Get the mean recall across labels

        :return: (``float``) mean recall
        """
        return np.mean(self.get_recall())

    def get_weighted_recall(self):
        return np.sum(self.get_recall() * self.get_support())/float(self.get_total())

    def get_weighted_f(self, beta=1):
        return np.sum(self.get_class_f(beta) * self.get_support())/float(self.get_total())

    def get_macro_f(self, beta=1):
        """Get the macro F_b, with adjustable beta (defaulting to F1)

        :param beta: (``float``) defaults to 1 (F1)
        :return: (``float``) macro F_b
        """
        if beta < 0:
            raise Exception('Beta must be greater than 0')
        return np.mean(self.get_class_f(beta))

    def get_class_f(self, beta=1):
        p = self.get_precision()
        r = self.get_recall()

        b = beta*beta
        d = (b * p + r)
        d = (d == 0) + d

        return (b + 1) * p * r / d

    def get_f(self, beta=1):
        """Get 2 class F_b, with adjustable beta (defaulting to F1)

        :param beta: (``float``) defaults to 1 (F1)
        :return: (``float``) 2-class F_b
        """
        p = self.get_precision()[1]
        r = self.get_recall()[1]
        if beta < 0:
            raise Exception('Beta must be greater than 0')
        d = (beta*beta * p + r)
        if d == 0:
            return 0
        return (beta*beta + 1) * p * r / d

    def get_all_metrics(self):
        """Make a map of metrics suitable for reporting, keyed by metric name

        :return: (``dict``) Map of metrics keyed by metric names
        """
        metrics = {'acc': self.get_acc()}
        # If 2 class, assume second class is positive AKA 1
        if len(self.labels) == 2:
            metrics['precision'] = self.get_precision()[1]
            metrics['recall'] = self.get_recall()[1]
            metrics['f1'] = self.get_f(1)
        else:
            metrics['mean_precision'] = self.get_mean_precision()
            metrics['mean_recall'] = self.get_mean_recall()
            metrics['macro_f1'] = self.get_macro_f(1)
            metrics['weighted_precision'] = self.get_weighted_precision()
            metrics['weighted_recall'] = self.get_weighted_recall()
            metrics['weighted_f1'] = self.get_weighted_f(1)
        return metrics

    def add_batch(self, truth, guess):
        """Add a batch of data to the confusion matrix

        :param truth: The truth tensor
        :param guess: The guess tensor
        :return:
        """
        for truth_i, guess_i in zip(truth, guess):
            self.add(truth_i, guess_i)


Our `Trainer` is simple, but it gets the job done.  We will use PyTorch's `DataLoader` to feed our batches to the trainer.  The `run()` method cycles a single epoch.
For every batch, we will do a stochastic gradient minibatch update, and we return the loss and the predictions and ground truth back to the `run()` method for tabulation

In [6]:

class Trainer:
    def __init__(self, optimizer: torch.optim.Optimizer):
        self.optimizer = optimizer

    def run(self, model, labels, train, loss, batch_size):
        model.train()
        train_loader = DataLoader(train, batch_size=batch_size, shuffle=True)

        cm = ConfusionMatrix(labels)

        for batch in train_loader:
            loss_value, y_pred, y_actual = self.update(model, loss, batch)
            _, best = y_pred.max(1)
            yt = y_actual.cpu().int().numpy()
            yp = best.cpu().int().numpy()
            cm.add_batch(yt, yp)

        print(cm.get_all_metrics())
        return cm

    def update(self, model, loss, batch):
        self.optimizer.zero_grad()
        x, lengths, y = batch
        lengths, perm_idx = lengths.sort(0, descending=True)
        x_sorted = x[perm_idx]
        y_sorted = y[perm_idx]
        y_sorted = y_sorted.to('cuda:0')
        inputs = (x_sorted.to('cuda:0'), lengths)
        y_pred = model(inputs)
        loss_value = loss(y_pred, y_sorted)
        loss_value.backward()
        self.optimizer.step()
        return loss_value.item(), y_pred, y_sorted

After training a epoch, we would like to test the validation performance.  Our evaluator class is similar to our `Trainer`, but it doesnt update our model -- it just gives us a way to evaluate the model on data

In [7]:

class Evaluator:
    def __init__(self):
        pass

    def run(self, model, labels, dataset, batch_size=1):
        model.eval()
        valid_loader = DataLoader(dataset, batch_size=batch_size)
        cm = ConfusionMatrix(labels)
        for batch in valid_loader:
            y_pred, y_actual = self.inference(model, batch)
            _, best = y_pred.max(1)
            yt = y_actual.cpu().int().numpy()
            yp = best.cpu().int().numpy()
            cm.add_batch(yt, yp)
        return cm

    def inference(self, model, batch):
        with torch.no_grad():
            x, lengths, y = batch
            lengths, perm_idx = lengths.sort(0, descending=True)
            x_sorted = x[perm_idx]
            y_sorted = y[perm_idx]
            y_sorted = y_sorted.to('cuda:0')
            inputs = (x_sorted.to('cuda:0'), lengths)
            y_pred = model(inputs)
            return y_pred, y_sorted


We can encapsulate training multiple epochs and testing in a single function.   The best model is defined in terms of some metric -- here accuracy, and we only save the checkpoints when we improve on the model.  This is called early stopping, and is particularly helpful on smaller datasets

In [8]:
def fit(model, labels, optimizer, loss, epochs, batch_size, train, valid, test):

    trainer = Trainer(optimizer)
    evaluator = Evaluator()
    best_acc = 0.0

    for epoch in range(epochs):
        print('EPOCH {}'.format(epoch + 1))
        print('=================================')
        print('Training Results')
        cm = trainer.run(model, labels, train, loss, batch_size)
        print('Validation Results')
        cm = evaluator.run(model, labels, valid)
        print(cm.get_all_metrics())
        if cm.get_acc() > best_acc:
            print('New best model {:.2f}'.format(cm.get_acc()))
            best_acc = cm.get_acc()
            torch.save(model.state_dict(), './checkpoint.pth')
    if test:
        model.load_state_dict(torch.load('./checkpoint.pth'))
        cm = evaluator.run(model, labels, test)
        print('Final result')
        print(cm.get_all_metrics())
    return cm.get_acc()

### A Reader for our Data

We need a reader to load our data from files and put it into a `Dataset`.

The reader needs to perform a few steps

* **read in sentences and labels**: it should convert the sentences into tokens and record a vocabulary of the labels
* **vectorize tokens**: it should convert tokens into tensors that comprise rows in our `TensorDataset`
* **tabulate the vocabulary**: if no vectorizer is provided, we need to build a vocab of attested words.  If a vectorizer is provided upfront, we dont need this step

In [9]:

def whitespace_tokenizer(words: str) -> List[str]:
    return words.split()

def sst2_tokenizer(words: str) -> List[str]:
    REPLACE = { "'s": " 's ",
                "'ve": " 've ",
                "n't": " n't ",
                "'re": " 're ",
                "'d": " 'd ",
                "'ll": " 'll ",
                ",": " , ",
                "!": " ! ",
                }
    words = words.lower()
    words = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", words)
    for k, v in REPLACE.items():
            words = words.replace(k, v)
    return [w.strip() for w in words.split()]


class Reader:

    def __init__(self, files, lowercase=True, min_freq=0,
                 tokenizer=sst2_tokenizer, vectorizer=None):
        self.lowercase = lowercase
        self.tokenizer = tokenizer
        build_vocab = vectorizer is None
        self.vectorizer = vectorizer if vectorizer else self._vectorizer
        x = Counter()
        y = Counter()
        for file_name in files:
            if file_name is None:
                continue
            with codecs.open(file_name, encoding='utf-8', mode='r') as f:
                for line in f:
                    words = line.split()
                    y.update(words[0])

                    if build_vocab:
                        words = self.tokenizer(' '.join(words[1:]))
                        words = words if not self.lowercase else [w.lower() for w in words]
                        x.update(words)
        self.labels = list(y.keys())

        if build_vocab:
            x = dict(filter(lambda cnt: cnt[1] >= min_freq, x.items()))
            alpha = list(x.keys())
            alpha.sort()
            self.vocab = {w: i+1 for i, w in enumerate(alpha)}
            self.vocab['[PAD]'] = 0

        self.labels.sort()

    def _vectorizer(self, words: List[str]) -> List[int]:
        return [self.vocab.get(w, 0) for w in words]

    def load(self, filename: str) -> TensorDataset:
        label2index = {l: i for i, l in enumerate(self.labels)}
        xs = []
        lengths = []
        ys = []
        with codecs.open(filename, encoding='utf-8', mode='r') as f:
            for line in f:
                words = line.split()
                ys.append(label2index[words[0]])
                words = self.tokenizer(' '.join(words[1:]))
                words = words if not self.lowercase else [w.lower() for w in words]
                vec = self.vectorizer(words)
                lengths.append(len(vec))
                xs.append(torch.tensor(vec, dtype=torch.long))
        x_tensor = torch.nn.utils.rnn.pad_sequence(xs, batch_first=True)
        lengths_tensor = torch.tensor(lengths, dtype=torch.long)
        y_tensor = torch.tensor(ys, dtype=torch.long)
        return TensorDataset(x_tensor, lengths_tensor, y_tensor)


### Pre-trained Embeddings

We would like to investigate how pre-training embeddings helps our models improve.  To do this, we need a mechanism to load in pre-trained embeddings and convert them into PyTorch's `nn.Embedding` object.  Specifically, we wish to support `word2vec`, `GloVe` and `fastText` embeddings. Rest-assured, these are simple file formats, and you do not need any 3rd party dependencies to read them in!  We will do it by hand.

For binary files, the first line contains 2 numbers delimited by a space.  The first number is the vocab size and the second is the embedding dimension.  We then read each line, splitting it by a space and reading the first portion as the vocabulary (token) and the second portion as a binary vector.

For text files, the first line may contain 2 numbers as in the binary file, but for `GloVe` files, this is omitted.  We can check if the first line contains the dimensions, and if it doesnt, we can just read in the first vector to figure out its dimension (again its space delimited, but the vector is also space delimited, so we split along the first space to find the token).

Notice that in this code, we have already created an alphabet that we will pass in for each key, so if that word is present in the embedding file, we will use its value, otherwise, we will initialize the vector randomly.

In [10]:
def init_embeddings(vocab_size, embed_dim, unif):
    return np.random.uniform(-unif, unif, (vocab_size, embed_dim))


class EmbeddingsReader:

    @staticmethod
    def from_text(filename, vocab, unif=0.25):

        with io.open(filename, "r", encoding="utf-8") as f:
            for i, line in enumerate(f):
                line = line.rstrip("\n ")
                values = line.split(" ")

                if i == 0:
                    # fastText style
                    if len(values) == 2:
                        weight = init_embeddings(len(vocab), values[1], unif)
                        continue
                    # glove style
                    else:
                        weight = init_embeddings(len(vocab), len(values[1:]), unif)
                word = values[0]
                if word in vocab:
                    vec = np.asarray(values[1:], dtype=np.float32)
                    weight[vocab[word]] = vec
        if '[PAD]' in vocab:
            weight[vocab['[PAD]']] = 0.0

        embeddings = nn.Embedding(weight.shape[0], weight.shape[1])
        embeddings.weight = nn.Parameter(torch.from_numpy(weight).float())
        return embeddings, weight.shape[1]

    @staticmethod
    def from_binary(filename, vocab, unif=0.25):
        def read_word(f):

            s = bytearray()
            ch = f.read(1)

            while ch != b' ':
                s.extend(ch)
                ch = f.read(1)
            s = s.decode('utf-8')
            # Only strip out normal space and \n not other spaces which are words.
            return s.strip(' \n')

        vocab_size = len(vocab)
        with io.open(filename, "rb") as f:
            header = f.readline()
            file_vocab_size, embed_dim = map(int, header.split())
            weight = init_embeddings(len(vocab), embed_dim, unif)
            if '[PAD]' in vocab:
                weight[vocab['[PAD]']] = 0.0
            width = 4 * embed_dim
            for i in range(file_vocab_size):
                word = read_word(f)
                raw = f.read(width)
                if word in vocab:
                    vec = np.fromstring(raw, dtype=np.float32)
                    weight[vocab[word]] = vec
        embeddings = nn.Embedding(weight.shape[0], weight.shape[1])
        embeddings.weight = nn.Parameter(torch.from_numpy(weight).float())
        return embeddings, embed_dim



### Now to run some stuff!

We did a lot of work to set things up, but its pretty boilerplate and we will reuse a lot of it.  So far, we made 2 classifiers we can run along with code to train and evaluate our models, and a reader to load our data.


In [11]:
BASE = 'sst2'
TRAIN = os.path.join(BASE, 'stsa.binary.phrases.train')
VALID = os.path.join(BASE, 'stsa.binary.dev')
TEST = os.path.join(BASE, 'stsa.binary.test')
PRETRAINED_EMBEDDINGS_FILE = 'GoogleNews-vectors-negative300.bin'



Lets read in our datasets:

In [12]:
r = Reader((TRAIN, VALID, TEST,))
train = r.load(TRAIN)
valid = r.load(VALID)
test = r.load(TEST)

## Model trained with randomly initialized embeddings

First, we are going to train a model without any pretrained embeddings for 10 epochs.  During training, we will see the training and validation performance, and after the final epoch, we will see the results from the best model trained on these epochs.

In [13]:
embed_dim = 300
embeddings = nn.Embedding(len(r.vocab), embed_dim)
model  = ConvClassifier(embeddings, len(r.labels), embed_dim)

num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Model has {num_params} parameters")


model.to('cuda:0')
loss = torch.nn.NLLLoss()
loss = loss.to('cuda:0')

learnable_params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.Adadelta(learnable_params, lr=1.0)

fit(model, r.labels, optimizer, loss, 10, 50, train, valid, test)


Model has 5442302 parameters
EPOCH 1
Training Results


<ipython-input-5-d4e4d1ff9cc0>:19: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self._cm = np.zeros((nc, nc), dtype=np.int)


{'acc': 0.5876093086108548, 'precision': 0.6142618214991638, 'recall': 0.6692065595494451, 'f1': 0.6405581100364675}
Validation Results
{'acc': 0.6674311926605505, 'precision': 0.8208333333333333, 'recall': 0.4436936936936937, 'f1': 0.5760233918128655}
New best model 0.67
EPOCH 2
Training Results
{'acc': 0.633229817699874, 'precision': 0.6503127945839403, 'recall': 0.718284862396176, 'f1': 0.6826108955979086}
Validation Results
{'acc': 0.6995412844036697, 'precision': 0.7286432160804021, 'recall': 0.6531531531531531, 'f1': 0.6888361045130641}
New best model 0.70
EPOCH 3
Training Results
{'acc': 0.6606333077792649, 'precision': 0.673503171020101, 'recall': 0.7413332071274759, 'f1': 0.7057922364656318}
Validation Results
{'acc': 0.7041284403669725, 'precision': 0.6608996539792388, 'recall': 0.8603603603603603, 'f1': 0.7475538160469667}
New best model 0.70
EPOCH 4
Training Results
{'acc': 0.6820597445459389, 'precision': 0.693000347161951, 'recall': 0.7557916656806835, 'f1': 0.72303530397

0.7287204832509611

Yikes, thats not very encouraging!  What about our LSTM?

In [14]:
embed_dim = 300
embeddings = nn.Embedding(len(r.vocab), embed_dim)
model  = LSTMClassifier(embeddings, len(r.labels), embed_dim, 100, hidden_units=[100])

num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Model has {num_params} parameters")


model.to('cuda:0')
loss = torch.nn.NLLLoss()
loss = loss.to('cuda:0')

learnable_params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.Adadelta(learnable_params, lr=1.0)

fit(model, r.labels, optimizer, loss, 10, 50, train, valid, test)


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
<ipython-input-5-d4e4d1ff9cc0>:19: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self._cm = np.zeros((nc, nc), dtype=np.int)


Model has 5342502 parameters
EPOCH 1
Training Results
{'acc': 0.6346201322747885, 'precision': 0.6417955352307599, 'recall': 0.757187817979602, 'f1': 0.6947327282991012}
Validation Results
{'acc': 0.7591743119266054, 'precision': 0.7839805825242718, 'recall': 0.7274774774774775, 'f1': 0.7546728971962617}
New best model 0.76
EPOCH 2
Training Results
{'acc': 0.7455724327906342, 'precision': 0.75547494592646, 'recall': 0.7934641141531982, 'f1': 0.7740036702330252}
Validation Results
{'acc': 0.7706422018348624, 'precision': 0.7606837606837606, 'recall': 0.8018018018018018, 'f1': 0.7807017543859649}
New best model 0.77
EPOCH 3
Training Results
{'acc': 0.7745351541689947, 'precision': 0.784944514357625, 'recall': 0.8118034028254336, 'f1': 0.7981480619794333}
Validation Results
{'acc': 0.7981651376146789, 'precision': 0.8160377358490566, 'recall': 0.7792792792792793, 'f1': 0.7972350230414746}
New best model 0.80
EPOCH 4
Training Results
{'acc': 0.7914917945452892, 'precision': 0.8023996308260

0.7946183415705657

## Same model with pre-trained word embeddings

The models below are identical to the ones above, the only difference is that we are going to initialize the embeddings using our previously defined `EmbeddingsReader`.  First lets take a look at our CNN model again.  Notice we only run 5 epochs here instead of 10!

In [15]:
embeddings, embed_dim = EmbeddingsReader.from_binary(PRETRAINED_EMBEDDINGS_FILE, r.vocab)
model  = ConvClassifier(embeddings, len(r.labels), embed_dim)
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Model has {num_params} parameters")


model.to('cuda:0')
loss = torch.nn.NLLLoss()
loss = loss.to('cuda:0')

learnable_params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.Adadelta(learnable_params, lr=1.0)

fit(model, r.labels, optimizer, loss, 5, 50, train, valid, test)

<ipython-input-10-c5a91f4ea3df>:60: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  vec = np.fromstring(raw, dtype=np.float32)


Model has 5442302 parameters
EPOCH 1
Training Results


<ipython-input-5-d4e4d1ff9cc0>:19: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self._cm = np.zeros((nc, nc), dtype=np.int)


{'acc': 0.8308753784384298, 'precision': 0.8393798017779635, 'recall': 0.8557467048439386, 'f1': 0.847484239881887}
Validation Results
{'acc': 0.8348623853211009, 'precision': 0.8807106598984772, 'recall': 0.7815315315315315, 'f1': 0.8281622911694511}
New best model 0.83
EPOCH 2
Training Results
{'acc': 0.879250529488962, 'precision': 0.8864531557722968, 'recall': 0.8946969876239381, 'f1': 0.8905559939230488}
Validation Results
{'acc': 0.8268348623853211, 'precision': 0.9150141643059491, 'recall': 0.7274774774774775, 'f1': 0.8105395232120451}
EPOCH 3
Training Results
{'acc': 0.8961811826769402, 'precision': 0.9015114235500878, 'recall': 0.9103859532880569, 'f1': 0.9059269550474486}
Validation Results
{'acc': 0.8451834862385321, 'precision': 0.8456375838926175, 'recall': 0.8513513513513513, 'f1': 0.8484848484848485}
New best model 0.85
EPOCH 4
Training Results
{'acc': 0.9056145320357064, 'precision': 0.9104600154824181, 'recall': 0.9184315767055539, 'f1': 0.9144284233342758}
Validation 

0.8687534321801208

Much better!  And now the LSTM!

In [16]:
model  = LSTMClassifier(embeddings, len(r.labels), embed_dim, 100, hidden_units=[100])

num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Model has {num_params} parameters")


model.to('cuda:0')
loss = torch.nn.NLLLoss()
loss = loss.to('cuda:0')

learnable_params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.Adadelta(learnable_params, lr=1.0)

fit(model, r.labels, optimizer, loss, 5, 50, train, valid, test)

Model has 5342502 parameters
EPOCH 1
Training Results


<ipython-input-5-d4e4d1ff9cc0>:19: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self._cm = np.zeros((nc, nc), dtype=np.int)


{'acc': 0.8801470874858695, 'precision': 0.8925727866904337, 'recall': 0.8886864336591022, 'f1': 0.8906253705504303}
Validation Results
{'acc': 0.8405963302752294, 'precision': 0.8105906313645621, 'recall': 0.8963963963963963, 'f1': 0.8513368983957218}
New best model 0.84
EPOCH 2
Training Results
{'acc': 0.9124361689686984, 'precision': 0.9250634244411469, 'recall': 0.9146217373813862, 'f1': 0.9198129484418319}
Validation Results
{'acc': 0.8623853211009175, 'precision': 0.8432203389830508, 'recall': 0.8963963963963963, 'f1': 0.868995633187773}
New best model 0.86
EPOCH 3
Training Results
{'acc': 0.9243253076233416, 'precision': 0.9344223202575415, 'recall': 0.9272580988665137, 'f1': 0.9308264246858445}
Validation Results
{'acc': 0.8727064220183486, 'precision': 0.8899297423887588, 'recall': 0.8558558558558559, 'f1': 0.8725602755453502}
New best model 0.87
EPOCH 4
Training Results
{'acc': 0.9327841374202518, 'precision': 0.9412268595631276, 'recall': 0.9360372938309, 'f1': 0.93862490360

0.8962108731466227

#### A quick note about these models on this data

Both of these models are surprisingly strong baselines and do fairly well on this dataset averaged over many runs.  Even with only 2-5 epochs of data it is quite common to see scores higher than in the Kim 2014 paper.

## Conclusions

### Its not hard to get good performance with a Deep Learning model for Text Classification

We saw above how to get good results on the SST-2 dataset using fairly simple deep learning models, even with very few training epochs.  This behavior is not limited to a single dataset -- these results have been shown over and over.  Also, using PyTorch, we were able to code an entire pipeline in this minimalistic notebook.

### Pre-trained embeddings often help

We can see that pre-trained embeddings can have a massive impact on the performance of our models, especially for smaller datasets.  The `word2vec` algorithm caused an explosion in the NLP community -- even though pre-training embeddings had been widely studied prior to that work, the results were reliable and fast.  `GloVe` and `fastText` embeddings came shortly thereafter, and all 3 models are in quite common use today.  The code above can load any of these flavors of embeddings and incorporate them into downstream models for large improvements.

For large datasets, like those used in Language Modeling and Neural Machine Translation, models are typically trained from random embeddings, which are sufficient in those cases.

### Incorporating pre-trained embeddings into your model is simple

The file formats are very simple to read and can be incorporated with only a few lines of code.  In some cases, memory-mapping the file can increase the loading speed.  This is implemented in [Baseline](https://github.com/dpressel/baseline/)

### Some Further Resources

- [The Unreasonable Effectiveness of Recurrent Neural Networks, Karpathy, 2015](https://karpathy.github.io/2015/05/21/rnn-effectiveness/)
- Tensorflow tutorial for word2vec: https://www.tensorflow.org/tutorials/representation/word2vec
- Tensorflow docs on feature columns (for images above): https://www.tensorflow.org/guide/feature_columns
- Xin Rong, wrote some amazing software to visualize word embeddings and the training process.  Sadly, Xin is no longer with us -- he was a great researcher and an awesome guy.  We miss him.
  - https://ronxin.github.io/wevi/
  - Accompanying talk from a2-dlearn 2015: https://www.youtube.com/channel/UCVdeq2cIxnujw2kTdzg2N5g
  - https://ronxin.github.io/lamvi/dist/#model=word2vec&backend=browser&query_in=darcy&query_out=G_bennet,B_circumstances
  - Accompanying paper for Lamvi [Visual Tools for Debugging Neural Language Models, Rong & Adar, 2016](http://www.cond.org/ICML16_NeuralVis.pdf)



**One thing I noticed with the both of these is that wether it had pre-read embeddings or not, the LSTM still had the higher final results score (if only slightly higher compared to the CNN model). One thing I want to test is how many epochs it will take until the new best model score is 1.00? After doing some research, I found that it might not be possible for such a thing to occur. Though I am still curious as to how I can call the fit funciton in the LSTM under the conditions that it goes on for an indefinet amount of epochs until it acheives a certain best model score?**

-Brian Mann